In [20]:
from file_encoder import FileEncoder
from data_manager import DataManager
from audio_utils import N_MELS
import numpy as np
import sys
import tensorflow as tf
from tensorflow.keras import layers, models, Model


TARGET_DIR = "../thirdparty/「波音リツ」歌声データベースVer2/DATABASE"
OUTPUT_DIR = "../master/ust/json"
    
PITCH_FEATURES = 1 # MIDI番号を0 ~ 100のスカラー
LYRIC_FEATURES = 1 # 歌詞をインデックスしたのでスカラー
DURATION_FEATURES = 1 # msの長さを正規化したのでスカラー
NOTE_CHUK_INDEX_FEATURES = 1 # 何番目の分割化を正規化したのでスカラー

LYRIC_INDEX_MAX_DIM = 150 # インデックス化した歌詞の最大インデックス（150種類くらい単語がありそう）
LYRIC_DIM = 64 # インデックス化した歌詞を埋め込みした後の次元
DURATION_DIM = 1 # 正規化した長さのミリ秒（0ミリ秒 ~ 10000ミリ秒（10秒）)

# TODO: N分割したいが今後考える（ビブラートなどを考慮するにはノートを更に分割したい）
NOTE_CHUNK = 1 # 1ノートに対して1分割する

    
pitch_input = layers.Input(shape=(NOTE_CHUNK,PITCH_FEATURES), name="pitch_input")
lyric_input = layers.Input(shape=(NOTE_CHUNK,), name="lyric_input")
duration_input = layers.Input(shape=(NOTE_CHUNK,DURATION_FEATURES), name="duration_input")
# note_chunk_index_input = layers.Input(shape=(NOTE_CHUNK,NOTE_CHUK_INDEX_FEATURES), name="note_chunk_index_input") # 何番目に分割したのか保持

# 歌詞を埋め込み
lyric_embedding = layers.Embedding(output_dim=LYRIC_DIM, input_dim=LYRIC_INDEX_MAX_DIM, name="lyric_embedding")(lyric_input) # 埋め込み

# 時系列
lstm_pitch = layers.LSTM(units=64, return_sequences=True)(pitch_input)
lstm_duration = layers.LSTM(units=64, return_sequences=True)(duration_input)
lstm_lyric_embedding = layers.LSTM(units=64, return_sequences=True)(lyric_embedding)
# lstm_note_chunk_index = layers.LSTM(units=64, return_sequences=True)(note_chunk_index_input)


merged = layers.Concatenate(axis=-1, name="merged_features")(
    [lstm_pitch, lstm_lyric_embedding, lstm_duration]
    # [lstm_pitch, lstm_lyric_embedding, lstm_duration, lstm_note_chunk_index]
)

# 最終的な時系列処理
final_lstm = layers.LSTM(128, return_sequences=True, name="final_lstm")(merged)

# 出力層（例: メルスペクトログラムへの回帰）
output = layers.Dense(N_MELS, activation="tanh", name="output")(final_lstm)



In [ ]:
89600 / 70